# SQL PROJECT- MUSIC STORE DATA ANALYSIS

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import mysql.connector

pd.set_option("display.width", 1000)

In [63]:
album = pd.read_csv(r"D:\SQL\SQL Project 2\music store data\album.csv")
artist = pd.read_csv(r"D:\SQL\SQL Project 2\music store data\artist.csv")
customers = pd.read_csv(r"D:\SQL\SQL Project 2\music store data\customers.csv")
employee = pd.read_csv(r"D:\SQL\SQL Project 2\music store data\employee.csv")
genre = pd.read_csv(r"D:\SQL\SQL Project 2\music store data\genre.csv")
invoice = pd.read_csv(r"D:\SQL\SQL Project 2\music store data\invoice.csv")
invoice_line = pd.read_csv(r"D:\SQL\SQL Project 2\music store data\invoice_line.csv")
media_type = pd.read_csv(r"D:\SQL\SQL Project 2\music store data\media_type.csv")
playlist = pd.read_csv(r"D:\SQL\SQL Project 2\music store data\playlist.csv")
playlist_track = pd.read_csv(r"D:\SQL\SQL Project 2\music store data\playlist_track.csv")
track = pd.read_csv(r"D:\SQL\SQL Project 2\music store data\track.csv")

In [3]:
db_user = "root"         
db_password = "1111"  
db_host = "localhost"    
db_name = "MusicDB" 

engine = create_engine(f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}")

In [ ]:
artist.to_sql("artist", con=engine, if_exists="append", index=False)

album.to_sql("album", con=engine, if_exists="append", index=False)

media_type.to_sql("mediatype", con=engine, if_exists="append", index=False)

genre.to_sql("genre", con=engine, if_exists="append", index=False)

track.to_sql("track", con=engine, if_exists="append", index=False)

playlist.to_sql("playlist", con=engine, if_exists="append", index=False)

playlist_track.to_sql("playlisttrack", con=engine, if_exists="append", index=False)

employee.to_sql("employee", con=engine, if_exists="append", index=False)

customers.to_sql("customers", con=engine, if_exists="append", index=False)

invoice.to_sql("invoice", con=engine, if_exists="append", index=False)

invoice_line.to_sql("invoiceline", con=engine, if_exists="append", index=False)


9

In [12]:
import pandas as pd

tables = ["Artist", "Album", "MediaType", "Genre", "Track", "Playlist", 
          "PlaylistTrack", "Employee", "Customers", "Invoice", "InvoiceLine"]

for t in tables:
    print(f"\nPreview of {t} table:")
    df_preview = pd.read_sql(f"SELECT * FROM {t} LIMIT 5;", con=engine)
    print(df_preview)


Preview of Artist table:
   artist_id               name
0          1              AC/DC
1          2             Accept
2          3          Aerosmith
3          4  Alanis Morissette
4          5    Alice In Chains

Preview of Album table:
   album_id                                  title  artist_id
0         1  For Those About To Rock We Salute You          1
1         2                      Balls to the Wall          2
2         3                      Restless and Wild          2
3         4                      Let There Be Rock          1
4         5                               Big Ones          3

Preview of MediaType table:
   media_type_id                         name
0              1              MPEG audio file
1              2     Protected AAC audio file
2              3  Protected MPEG-4 video file
3              4     Purchased AAC audio file
4              5               AAC audio file

Preview of Genre table:
   genre_id                name
0         1            

# Question Set 1 - Easy

In [14]:
# Who is the senior most employee based on job title?

EMP_table = "employee"
df = pd.read_sql(f"SELECT first_name, last_name, title FROM {EMP_table} ORDER BY title DESC LIMIT 1;", con=engine)
print(df)

  first_name last_name                   title
0      Mohan     Madan  Senior General Manager


In [15]:
# Which countries have the most Invoices?

INV_table = "invoice"
df = pd.read_sql(f"SELECT billing_country, count(billing_country) AS country_count FROM {INV_table} GROUP BY billing_country ORDER BY country_count DESC;", con=engine)
print(df)

   billing_country  country_count
0              USA            131
1           Canada             76
2           Brazil             61
3           France             50
4          Germany             41
5   Czech Republic             30
6         Portugal             29
7   United Kingdom             28
8            India             21
9          Ireland             13
10           Chile             13
11         Finland             11
12           Spain             11
13          Poland             10
14         Denmark             10
15       Australia             10
16         Hungary             10
17          Sweden             10
18     Netherlands             10
19          Norway              9
20           Italy              9
21         Austria              9
22         Belgium              7
23       Argentina              5


In [ ]:
# What are the top 3 values of the total invoice?

df = pd.read_sql(f"SELECT * FROM invoice ORDER BY total DESC LIMIT 3;", con=engine)
print(df)

   invoice_id  customer_id invoice_date         billing_address billing_city billing_state billing_country billing_postal_code  total
0         183           42   2018-02-09  9, Place Louis Barthou     Bordeaux          None          France               33000  23.76
1          92           32   2017-07-02      696 Osborne Street     Winnipeg            MB          Canada             R3L 2B9  19.80
2          31            3   2017-02-21       1498 rue Bélanger     Montréal            QC          Canada             H2G 1A7  19.80


In [ ]:
# Which city has the best customers? We would like to throw a promotional Music Festival in the city we made the most money. 
# Write a query that returns one city that has the highest sum of invoice totals. Return both the city name & sum of all invoice totals

df = pd.read_sql(f"SELECT billing_city, sum(total) AS total_sales FROM invoice GROUP BY billing_city ORDER BY total_sales DESC LIMIT 1;", con=engine)
print(df)

  billing_city  total_sales
0       Prague       273.24


In [27]:
# Who is the best customer? The customer who has spent the most money will be declared the best customer.
#  Write a query that returns the person who has spent the most money 

df = pd.read_sql(f"SELECT c.customer_id, c.first_name, c.last_name, SUM(i.total) AS total_spent FROM customers c JOIN invoice i ON c.customer_id = i.customer_id GROUP BY c.customer_id, c.first_name, c.last_name ORDER BY total_spent DESC LIMIT 1;", con=engine)
print(df)

   customer_id first_name    last_name  total_spent
0            5  František  Wichterlová       144.54


# Question Set 2 – Moderate

In [28]:
# Write a query to return the email, first name, last name, & Genre of all Rock Music listeners. 
# Return your list ordered alphabetically by email starting with A

df = pd.read_sql(f"""SELECT DISTINCT c.email, c.first_name, c.last_name, g.name AS genre FROM customers c
                 JOIN invoice I ON c.customer_id = i.customer_id JOIN invoice_line il ON i.invoice_id = il.invoice_id
                 JOIN track t ON il.track_id = t.track_id JOIN genre g ON t.genre_id = g.genre_id
                 WHERE g.name = 'Rock' ORDER BY c.email ASC;""", con=engine)
print(df)

Empty DataFrame
Columns: [email, first_name, last_name, genre]
Index: []


In [30]:
#Let's invite the artists who have written the most rock music in our dataset. Write a query that returns the Artist name and 
# total track count of the top 10 rock bands

df = pd.read_sql(f"""SELECT ar.name AS artist, COUNT(t.track_id) AS total_tracks FROM artist ar JOIN album al ON ar.artist_id = al.artist_id
                JOIN track t ON al.album_id = t.album_id JOIN genre g ON t.genre_id = g.genre_id WHERE g.name = 'Rock'
                GROUP BY ar.artist_id, ar.name ORDER BY total_tracks DESC LIMIT 10;""", con=engine)
print(df)

                         artist  total_tracks
0                  Led Zeppelin           114
1                            U2           112
2                   Deep Purple            92
3                   Iron Maiden            81
4                     Pearl Jam            54
5                     Van Halen            52
6                         Queen            45
7            The Rolling Stones            41
8  Creedence Clearwater Revival            40
9                          Kiss            35


In [34]:
# Return all the track names that have a song length longer than the average song length. Return the Name and Milliseconds for each track. 
# Order by the song length with the longest songs listed first

df = pd.read_sql("SELECT name, milliseconds FROM track WHERE milliseconds > (SELECT AVG(milliseconds) FROM track) order by milliseconds DESC;", con=engine)
print(df)

                                                  name  milliseconds
0                               Occupation / Precipice       5286953
1                              Through a Looking Glass       5088838
2                          Greetings from Earth, Pt. 1       2960293
3                              The Man With Nine Lives       2956998
4                          Battlestar Galactica, Pt. 2       2956081
..                                                 ...           ...
489                                   22 Acacia Avenue        395572
490                                  The Unforgiven II        395520
491                                 The Shortest Straw        395389
492  Concerto for Clarinet in A Major, K. 622: II. ...        394482
493                                        Wicked Ways        393691

[494 rows x 2 columns]
